## Setup

The following code sets up requirements, imports, and helper functions (you can ignore this).

In [6]:
!pip install gym-minigrid &>/dev/null

In [7]:
!rm uaas -rf
!git clone https://github.com/kehemo/uaas

Cloning into 'uaas'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 114 (delta 57), reused 62 (delta 24), pack-reused 0
Receiving objects: 100% (114/114), 1.42 MiB | 8.45 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [9]:
from uaas.foo import *
from uaas.cartpole import *
from uaas.uaas import *
import sys

smooth_reward_window = 50
args = Config(
    use_critic=False,
    max_episodes=1000,
    smooth_reward_window=smooth_reward_window,
    lr=1e-2,
    use_gae=False,
)
env = make_env()
acmodel = Policy(env.action_space.n)
df = run_experiment(
    env,
    acmodel,
    preprocess_obss,
    ["return_per_episode"],
    args,
    update_parameters_reinforce,
)


  0%|          | 0/1000 [00:00<?, ?it/s]


AttributeError: 'Tensor' object has no attribute 'sample'